In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Cargar dataset limpio
df = pd.read_csv("./data/dataset_ready/movimientos_limpio.csv")
print("Dataset limpio cargado:", df.shape)


Dataset limpio cargado: (13361, 136)


In [2]:
def calc_angle(a, b, c):
    """
    Calcula el ángulo entre tres puntos: a (proximal), b (vértice), c (distal)
    Devuelve el ángulo en grados.
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)


In [3]:
def extract_features_from_row(row):
    features = {}

    # Puntos clave
    hip_left = [row['x23'], row['y23']]
    knee_left = [row['x25'], row['y25']]
    ankle_left = [row['x27'], row['y27']]

    hip_right = [row['x24'], row['y24']]
    knee_right = [row['x26'], row['y26']]
    ankle_right = [row['x28'], row['y28']]

    shoulder_left = [row['x11'], row['y11']]
    shoulder_right = [row['x12'], row['y12']]

    # Ángulos
    features['angle_knee_left'] = calc_angle(hip_left, knee_left, ankle_left)
    features['angle_knee_right'] = calc_angle(hip_right, knee_right, ankle_right)
    features['angle_hip_left'] = calc_angle(shoulder_left, hip_left, knee_left)
    features['angle_hip_right'] = calc_angle(shoulder_right, hip_right, knee_right)

    # Inclinación del tronco (hombros vs caderas)
    trunk_vector = np.array(shoulder_right) + np.array(shoulder_left) - np.array(hip_right) - np.array(hip_left)
    features['trunk_inclination'] = np.arctan2(trunk_vector[1], trunk_vector[0]) * 180 / np.pi

    # Distancias
    features['shoulder_dist'] = np.linalg.norm(np.array(shoulder_left) - np.array(shoulder_right))
    features['hip_dist'] = np.linalg.norm(np.array(hip_left) - np.array(hip_right))

    return features


In [4]:
# Aplicar a cada fila del dataset
feature_rows = []
for idx, row in df.iterrows():
    feats = extract_features_from_row(row)
    feats['label'] = row['label']
    feats['class_name'] = row['class_name']
    feature_rows.append(feats)

df_features = pd.DataFrame(feature_rows)
print("✅ Dataset de características generado:", df_features.shape)
df_features.head()


✅ Dataset de características generado: (13361, 9)


,angle_knee_left,angle_knee_right,angle_hip_left,angle_hip_right,trunk_inclination,shoulder_dist,hip_dist,label,class_name
0,169.545252,165.763517,158.572490,160.627174,-85.881337,0.201857,0.116495,0,caminar_adelante
1,169.888034,166.247129,159.218057,160.725281,-86.048648,0.202016,0.117348,0,caminar_adelante
2,170.120438,166.575945,159.700908,160.617155,-86.237146,0.202360,0.117883,0,caminar_adelante
3,170.104022,166.921633,159.736942,160.517138,-86.344148,0.202853,0.117882,0,caminar_adelante
4,170.115277,167.128534,159.849440,160.381507,-86.480708,0.202973,0.117787,0,caminar_adelante


In [5]:
df_features.to_csv("./data/dataset_ready/features.csv", index=False)
print("Características guardadas.")


Características guardadas.
